In [ ]:
'''
[주의] 
!!!! 아직 dropout_data 만드는 중! 어디까지 만들었는지 확인하고 코드 돌리세요 존재 안하는 폴더 있을수도 !!!!
!!!!!dropout_data의 X는 sparse array, int32으로 통일했습니다.!!!!!!
!!!! adata 모두 normalize/log 했습니다. 전에 없던 오류가 생겼으면 얘가 원인일수도... !!!
봐서 알겠지만 원본/dropout 파일들의 이름이 같습니다. 코딩하기 편하려고 통일했으니
아무쪼록 헷갈리지 마시길...
dropout_data에 layers/dropped가 추가되어있습니다. X랑 같은 모양으로 boolean 
형식이고, True인 데이터들이 drop 후 0으로 채운 위치입니다. 혹시 모델이 NaN을
더 잘 impute한다면 이거 이용해서 데이터 수정하시면 됩니다.
모델 별로 이 노트북을 새로 복제해서 만들어두면 나중에 배포용 코드 만들기 편하겠죠?
이미지 파일이 필요하시면 hest_data/wsis 에서 가져오셔야 합니다. SPROD에 제가 만들어둔 가져오는
코드가 있긴 하니 필요하시면 가져가시죠.
이왕이면 dropout_data 폴더는 수정하지 말아주세요 (output 저장 포함)

결과는 bench_results에 저장됩니다. 혹시 잘못 돌려서 데이터 삭제해야하면 수동으로 들어가서 수정하시죠 ㅎ

[수정] 이라고 코멘트 해 둔 부분 적절히 고치시면 됩니다.
'''

패키지 업로드

In [1]:
import anndata as ad
import os
import numpy as np
from sklearn.metrics import adjusted_rand_score
import torch
from scipy.sparse import csr_matrix
import scanpy as sc


#[수정] 이 밑으로 [여기까지] 지우고 여러분 모델에 맞는 패키지 불러오시면 됩니다.
import pandas as pd
from tifffile import TiffFile, imwrite
import shutil
#[여기까지]

벤치마킹 데이터/환경 세팅

In [2]:
bench_data_dir = '/home/s2022310812/bench_data' # bench_data 경로 [수정]
dropout_data_dir = '/home/s2022310812/dropout_data' #dropout_data 경로 [수정]
datasets = [ #[수정]
    "IDC",
  # "COAD",
  # "READ",
  # "LYMPH_IDC",
]
model = "SPROD" #[수정] 모델명
percents = [ #[수정] dropout percentage
    # '10%',
    #'20%',
    '30%'
] 
samples = [ #[수정] dropout percentage data version
    '1',
    # '2',
    #'3',
    #'4',
    #'5'
] 

trial = '1' #[수정] dropout percentage data version trial number (1-5)
man_sc = {}
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu') #[수정] cuda:사용할 GPU

벤치마킹 모델 구동

In [3]:
wsis_dir = '/home/s2022310812/hest_data/wsis'

for dataset in datasets:
    for percent in percents:
        for sample in samples:
            bench_path = os.path.join(bench_data_dir,dataset,'adata')
            dropout_path = os.path.join(dropout_data_dir,dataset,percent+"-"+sample)
            
            adata_names = [f for f in os.listdir(bench_path) if os.path.isfile(os.path.join(bench_path, f))]
        
            for adata_name in adata_names:

                adata = ad.read_h5ad(os.path.join(bench_path,adata_name))
                sc.pp.normalize_total(adata, target_sum=1e6, exclude_highly_expressed= False) 
                sc.pp.log1p(adata)
                sc.pp.scale(adata)
                
                adata_drop = ad.read_h5ad(os.path.join(dropout_path,adata_name))
                sc.pp.normalize_total(adata_drop, target_sum=1e6, exclude_highly_expressed= False)
                sc.pp.log1p(adata_drop)
                sc.pp.scale(adata_drop)
                dropped_mask = torch.tensor(adata_drop.layers['dropped'], dtype=torch.bool, device=device)
                
                """
                [수정]
                여기서부터 [여기까지] adata_drop.X에 본인 모델 적용시켜서 adata2에 저장해주시면 됩니다.
                [주의] adata2에 ""adata.X 에 해당하는 파일"" 저장해주세요... 그냥 그렇게 만들었어요..
                예시로 제 코드 보여드림. 여러분한테는 쓸모가 없겠죠? 지우고 시작하시면 됩니다 화이팅
                """
                # Define the custom directory paths
                convert_dir = os.path.join("/home/s2022310812/SPROD/bench_test",percent+"-"+sample, adata_name, 'conv')
                out_dir = os.path.join("/home/s2022310812/SPROD/bench_test",percent+"-"+sample, adata_name, 'out')
        
                # Ensure the directories exist
                os.makedirs(convert_dir, exist_ok=True)
                os.makedirs(out_dir, exist_ok=True)
        
                # Remove the .h5ad extension from adata_name to get the base name (e.g., "aaa")
                base_name = adata_name.replace(".h5ad", "")
        
                # Define the path for the .tif file and the convert_dir
                tif_file = os.path.join(wsis_dir, f"{base_name}.tif")
                tif_target_file = os.path.join(convert_dir, f"{base_name}.tif")
        
                # Check if the .tif file exists and copy it to the convert_dir if it does
        
                def copy_highest_resolution_tif(tif_file, tif_target_file):
                    if os.path.exists(tif_file):
                        if not os.path.exists(tif_target_file):  # Check if the .tif file is already in the target directory
                            try:
                                with TiffFile(tif_file) as tif:
                                    # Extract the highest resolution (first series/page)
                                    highest_res_image = tif.pages[0].asarray()
                                    print(tif.pages[0].asarray().shape)
        
                                    # Save the highest resolution to the target location
                                    imwrite(tif_target_file, highest_res_image)
                                    print(f"Copied highest resolution of {tif_file}")
                            except Exception as e:
                                print(f"Error processing {tif_file}: {e}")
                        else:
                            print(f"{tif_target_file} already exists. Skipping copy.")
                    else:
                        print(f"File {tif_file} does not exist.")
                if not base_name == "TENX99": 
                    copy_highest_resolution_tif(tif_file, tif_target_file)
                if base_name == "TENX99":
                    continue
                # Check if adata_drop.X is sparse or dense, and convert accordingly
                if isinstance(adata_drop.X, csr_matrix):  # Sparse matrix
                    expression_matrix = pd.DataFrame(adata_drop.X.toarray(), index=adata_drop.obs.index, columns=adata_drop.var.index)
                    is_sparse = True
                else:  # Dense matrix (numpy ndarray)
                    expression_matrix = pd.DataFrame(adata_drop.X, index=adata_drop.obs.index, columns=adata_drop.var.index)
                    is_sparse = False
        
                expression_matrix.index.name = ""  # Remove index name for cleaner output
        
                # Extract spatial coordinates for Spot_metadata.csv
                spatial_coords = pd.DataFrame(adata_drop.obsm["spatial"], columns=["X", "Y"])
                spatial_coords["X"], spatial_coords["Y"] = spatial_coords["Y"].copy(), spatial_coords["X"].copy()
                spatial_coords.index = adata_drop.obs.index  # Match indices with the expression matrix
        
                # Calculate the horizontal and vertical distances between the first two spots
                x_diff = abs(adata_drop.obs['pxl_col_in_fullres'].iloc[1] - adata_drop.obs['pxl_col_in_fullres'].iloc[0])
                y_diff = abs(adata_drop.obs['pxl_row_in_fullres'].iloc[1] - adata_drop.obs['pxl_row_in_fullres'].iloc[0])
        
                # Estimate the spot radius using the maximum of the two distances
                spot_radius = max(x_diff, y_diff) / 2  # Use the maximum of the two distances as the radius
        
                # Set the Spot_radius for each entry (you can apply this value uniformly or adjust as needed)
                spatial_coords["Spot_radius"] = spot_radius
        
                # Check if the files already exist and only create them if they don't
                counts_file = os.path.join(convert_dir, "Counts.txt")
                spot_metadata_file = os.path.join(convert_dir, "Spot_metadata.csv")
        
                if not os.path.exists(counts_file):
                    expression_matrix.to_csv(counts_file, sep="\t")
                    print(f"Created {counts_file}")
                else:
                    print(f"{counts_file} already exists. Skipping creation.")
        
                if not os.path.exists(spot_metadata_file):
                    spatial_coords.to_csv(spot_metadata_file, index=True, index_label="", header=True)
                    print(f"Created {spot_metadata_file}")
                else:
                    print(f"{spot_metadata_file} already exists. Skipping creation.")
        
                denoised_matrix_file = os.path.join(out_dir, 'sprod_Denoised_matrix.txt')  # Ensure the correct file name
                if not os.path.exists(denoised_matrix_file):
                    denoised_matrix_file = os.path.join(out_dir, 'denoised_stitched.txt')  # Ensure the correct file name
                
        
                # Run the sprod.py script
                if not os.path.exists(denoised_matrix_file):
                    if base_name =="NCBI783" or base_name=="NCBI785":
                        !python /home/s2022310812/SPROD/sprod.py {convert_dir} {out_dir} -y "single"
                    else:
                        !python /home/s2022310812/SPROD/sprod.py {convert_dir} {out_dir} -y "batch"
                
                denoised_matrix_file = os.path.join(out_dir, 'sprod_Denoised_matrix.txt')  # Ensure the correct file name
                if not os.path.exists(denoised_matrix_file):
                    denoised_matrix_file = os.path.join(out_dir, 'denoised_stitched.txt')  # Ensure the correct file name
                
        
                # Load the output denoised matrix
                counts = pd.read_csv(denoised_matrix_file, sep="\t", index_col=0)
        
                # Store the denoised matrix in adata2
                if is_sparse:
                    adata2 = csr_matrix(counts.values)  # Convert back to sparse matrix (csr_matrix)
                else:
                    adata2 = counts.values  # Keep it as dense numpy array

                """
                [여기까지]
                """
                
                
                
                # Step 1: Convert adata.X to float32 or int32 before creating the PyTorch tensor
                if isinstance(adata.X, np.ndarray):
                    # Convert to float32 if data is uint16 or other unsupported types
                    adata_X = adata.X.astype(np.float32) if adata.X.dtype == np.uint16 else adata.X.astype(np.float32)
                    adata_tensor = torch.tensor(adata_X, dtype=torch.float32, device=device)
                elif isinstance(adata.X, csr_matrix):
                    adata_X = adata.X.toarray().astype(np.float32)
                    adata_tensor = torch.tensor(adata_X, dtype=torch.float32, device=device)
                else:
                    print('adata.X 데이터 형식 확인해주세요')
                    
                # Step 2: Convert adata2 to float32 or int32 before creating the PyTorch tensor
                if isinstance(adata2, np.ndarray):
                    # Convert to float32 if data is uint16 or other unsupported types
                    adata2 = adata2.astype(np.float32) if adata2.dtype == np.uint16 else adata2.astype(np.float32)
                    adata2_tensor = torch.tensor(adata2, dtype=torch.float32, device=device)
                elif isinstance(adata2, csr_matrix):
                    adata2 = adata2.toarray().astype(np.float32)
                    adata2_tensor = torch.tensor(adata2, dtype=torch.float32, device=device)
                else:
                    print('adata2 데이터 형식 확인해주세요')
            
                # Step 3: Get the indices of the dropped values (where the mask is True)
                dropped_indices = dropped_mask.nonzero(as_tuple=True)
                
                # Step 4: Extract the corresponding values for dropped spots
                adata_values = adata_tensor[dropped_indices]
                adata2_values = adata2_tensor[dropped_indices]
                
                # Step 5: Convert to CPU and calculate manh.
                adata_values_cpu = adata_values.cpu().numpy()
                adata2_values_cpu = adata2_values.cpu().numpy()
                #print(adata_values_cpu[:10])
                #print(adata2_values_cpu[:10])
                     
                manhattan_distance = np.abs(adata_values_cpu - adata2_values_cpu).sum()
                num_values = adata_values_cpu.size
                manhattan_score = manhattan_distance / num_values
                
                # Store the result in the dictionary
                man_sc[dataset + "/" + adata_name] = manhattan_score
                
                print(man_sc)

            row_name = "Trial : " + trial
            my_dir = os.path.join('/home/s2022310812/bench_results', model,dataset)
            csv_path = os.path.join(my_dir, percent+"-"+sample +"_result.csv")
            # Ensure the directory exists
            os.makedirs(my_dir, exist_ok=True)
            
            # Extract column names and values
            columns = [key.split("/")[-1].split(".")[0] for key in man_sc.keys()]
            values = list(man_sc.values())
            
            # Create a DataFrame for the new row
            data = pd.DataFrame([values], columns=columns, index=[row_name])
            
            # Check if the file exists
            if os.path.exists(csv_path):
                # Read the existing CSV
                existing_data = pd.read_csv(csv_path, index_col=0)
            
                # Append the new row
                updated_data = pd.concat([existing_data, data])
            else:
                # If the file doesn't exist, use the new DataFrame
                updated_data = data
            
            # Save back to CSV
            updated_data.to_csv(csv_path) # [수정] 파라미터 최적화 등 결과 저장하지 않고 돌려보는 단계면 이 줄 주석처리


/home/s2022310812/.conda/envs/sprod2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/home/s2022310812/.conda/envs/sprod2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


/home/s2022310812/SPROD/bench_test/30%-1/NCBI785.h5ad/conv/NCBI785.tif already exists. Skipping copy.
/home/s2022310812/SPROD/bench_test/30%-1/NCBI785.h5ad/conv/Counts.txt already exists. Skipping creation.
/home/s2022310812/SPROD/bench_test/30%-1/NCBI785.h5ad/conv/Spot_metadata.csv already exists. Skipping creation.
{'IDC/NCBI785.h5ad': 1.0926849639570606}
/home/s2022310812/SPROD/bench_test/30%-1/NCBI783.h5ad/conv/NCBI783.tif already exists. Skipping copy.
/home/s2022310812/SPROD/bench_test/30%-1/NCBI783.h5ad/conv/Counts.txt already exists. Skipping creation.
/home/s2022310812/SPROD/bench_test/30%-1/NCBI783.h5ad/conv/Spot_metadata.csv already exists. Skipping creation.


/home/s2022310812/.conda/envs/sprod2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


{'IDC/NCBI785.h5ad': 1.0926849639570606, 'IDC/NCBI783.h5ad': 1.3124477419668465}


/home/s2022310812/.conda/envs/sprod2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/home/s2022310812/.conda/envs/sprod2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


/home/s2022310812/SPROD/bench_test/30%-1/TENX95.h5ad/conv/TENX95.tif already exists. Skipping copy.
/home/s2022310812/SPROD/bench_test/30%-1/TENX95.h5ad/conv/Counts.txt already exists. Skipping creation.
/home/s2022310812/SPROD/bench_test/30%-1/TENX95.h5ad/conv/Spot_metadata.csv already exists. Skipping creation.
{'IDC/NCBI785.h5ad': 1.0926849639570606, 'IDC/NCBI783.h5ad': 1.3124477419668465, 'IDC/TENX95.h5ad': 1.4379274056360023}


/home/s2022310812/.conda/envs/sprod2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
/home/s2022310812/.conda/envs/sprod2/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))
